# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/city_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kurāra,25.9833,79.9833,93.43,13,0,2.62,IN,1650938562
1,1,Rikitea,-23.1203,-134.9692,75.94,74,96,19.35,PF,1650938563
2,2,Kruisfontein,-34.0033,24.7314,48.88,84,0,7.25,ZA,1650938564
3,3,Cape Town,-33.9258,18.4232,60.91,73,0,11.01,ZA,1650938312
4,4,Caravelas,-17.7125,-39.2481,75.54,83,0,4.03,BR,1650938564


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Caravelas,-17.7125,-39.2481,75.54,83,0,4.03,BR,1650938564
51,51,Saint George,37.1041,-113.5841,76.24,10,0,4.61,US,1650938444
58,58,Tomatlán,19.9333,-105.2500,72.64,79,0,3.83,MX,1650938579
85,85,Kiryat Gat,31.6100,34.7642,72.39,76,0,2.24,IL,1650938526
88,88,Seoul,37.5683,126.9778,74.59,58,0,5.75,KR,1650938205
97,97,Destin,30.3935,-86.4958,72.97,91,0,5.01,US,1650938357
194,194,Xinzhi,36.4989,111.7047,79.86,23,0,9.10,CN,1650938616
216,216,Saint-Pierre,-21.3393,55.4781,71.28,75,0,8.05,RE,1650938372
289,289,Aragarças,-15.8975,-52.2508,75.88,70,0,0.67,BR,1650938642
305,305,Rincon,32.2960,-81.2354,70.84,83,0,6.91,US,1650938647


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 4: Caravelas.
Closest hotel is Pousada dos Navegantes.
------------
Retrieving Results for Index 51: Saint George.
Closest hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 58: Tomatlán.
Closest hotel is El Salvial.
------------
Retrieving Results for Index 85: Kiryat Gat.
Closest hotel is OlusHome.
------------
Retrieving Results for Index 88: Seoul.
Closest hotel is 롯데호텔 서울.
------------
Retrieving Results for Index 97: Destin.
Closest hotel is The Inn on Destin Harbor.
------------
Retrieving Results for Index 194: Xinzhi.
Closest hotel is Mingyue Hotel.
------------
Retrieving Results for Index 216: Saint-Pierre.
Closest hotel is Lindsey Hôtel.
------------
Retrieving Results for Index 289: Aragarças.
Closest hotel is Pousada Tropical.
------------
Retrieving Results for Index 305: Rincon.
Closest hotel is Days Inn Rincon.
------------
Retrieving Results for Index 354: Barão de Melgaço.
Closest hotel is Tuiuiu Pantanal Hote

In [6]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,Caravelas,-17.7125,-39.2481,75.54,83,0,4.03,BR,1650938564,Pousada dos Navegantes
51,51,Saint George,37.1041,-113.5841,76.24,10,0,4.61,US,1650938444,Best Western Plus Abbey Inn
58,58,Tomatlán,19.9333,-105.2500,72.64,79,0,3.83,MX,1650938579,El Salvial
85,85,Kiryat Gat,31.6100,34.7642,72.39,76,0,2.24,IL,1650938526,OlusHome
88,88,Seoul,37.5683,126.9778,74.59,58,0,5.75,KR,1650938205,롯데호텔 서울
97,97,Destin,30.3935,-86.4958,72.97,91,0,5.01,US,1650938357,The Inn on Destin Harbor
194,194,Xinzhi,36.4989,111.7047,79.86,23,0,9.10,CN,1650938616,Mingyue Hotel
216,216,Saint-Pierre,-21.3393,55.4781,71.28,75,0,8.05,RE,1650938372,Lindsey Hôtel
289,289,Aragarças,-15.8975,-52.2508,75.88,70,0,0.67,BR,1650938642,Pousada Tropical
305,305,Rincon,32.2960,-81.2354,70.84,83,0,6.91,US,1650938647,Days Inn Rincon


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))